# <font color='dimgrey'>Exchanges & multiple queues : FanOut Exchange </font>

## <font color='grey'> Till now ... </font>
In the last tutorial 'Task Queues' We had producer producing multiple tasks to queues and the task being divided among the consumers

Well this tutorial is all about
> [Published log messages are going to be broadcast to all the receivers](https://www.rabbitmq.com/tutorials/tutorial-three-python.html#:~:text=published%20log%20messages%20are%20going%20to%20be%20broadcast%20to%20all%20the%20receivers)

### <font color='dimgrey'>In the last tutorial we had a producer/publisher pushing messages to a default exchange and that exchange sending the message to declared queues and workers getting assigned the task parallely </font>

> <font color='dimgrey'> In this tutorial we are going to have a publisher sending messages to a specific exchange and that exchange pushing messages to all the queues in that exchange </font>

## <font color='dimgrey'> Exchange</font>
><t> <font color='dimgrey'> Essentially its decoupling publisher and queues. It recieves messages from publishers and pushes to queues. </font>
    
><t> <font color='dimgrey'> There are multiple types of exchanges and in this tutorial we will learn about 'fanout' type. It has a publisher which sends messages to the exchange which broadcasts all the messages to all the queues in that exchange</font>

###  <font color='dimgrey'> Last time we used the default exchange, here we will declare one first both while publishing and consuming </font>

In [ ]:
channel.exchange_declare(exchange='logs',
                         exchange_type='fanout')

>  <font color='dimgrey'> This step is important since publishing to a non-existtent exchange is forbidden </font> <br>

###  <font color='dimgrey'> And when we publish a message we will mention the exchange </font>

In [ ]:
channel.basic_publish(exchange='logs',
                      routing_key='',
                      body=message)

>  <font color='dimgrey'> In fanout exchange type we don't need to specify the 'routing_key' since every queue subscribing to this exchange gets the messages/tasks</font> <br>


###  <font color='dimgrey'> Consuming </font> <br>
> <font color='dimgrey'> While consuming also you need to declare the exchange with same name, but you don't need to specify a queue name now and we can keep that info abstract </font> <br>

####  <font color='dimgrey'> We can do just that by not specifing a name on the queue declaration </font> <br>


In [ ]:
result = channel.queue_declare(queue='')


####  <font color='dimgrey'> Also we don't need to persist the messages in the queue once the consumer connection is closed. Use the exclusive flag to make this queue exclusively for that consumer </font> <br>


In [ ]:
result = channel.queue_declare(queue='', exclusive=True)

####  <font color='dimgrey'> Also 'result.method.queue' will have the queue name value in it </font> <br>
> <font color='dimgrey'> "We will use that to bind the exchange to the queue" </font>

###  <font color='dimgrey'> Binding </font> <br>
> <font color='dimgrey'> Earlier queues were subscribed to the default exchange now we need to specify the exchange the queue is gonna subscribe to</font>

In [ ]:
channel.queue_bind(exchange='logs',
                   queue=result.method.queue)

####  <font color='dimgrey'> From now on all the tasks/messages in 'logs' exchange would get pushed to all the queues that binds to it </font> 


## <font color='dimgrey'> Let's put it all together - Publisher </font> 

In [3]:
# Emit_logs
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='logs', exchange_type='fanout')
def sendQueue():
    try:
        while True:
            # keep taking input till kernel is interrupted
            message = input("Enter new task")
            channel.basic_publish(exchange='logs', routing_key='', body=message)
            print(" [x] Sent %r" % message)
    except KeyboardInterrupt:
        # Exit gracefully
        connection.close()
sendQueue()

Enter new taskm .
 [x] Sent 'm .'
Enter new taskm ..
 [x] Sent 'm ..'
Enter new taskm ...
 [x] Sent 'm ...'


> <font color='dimgrey'> If no queues are binded to the exchange yet, the messages gets discarded </font> 

## <font color='dimgrey'> Let's put it all together - Consumer </font> 

In [ ]:
import pika

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='logs', exchange_type='fanout')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

channel.queue_bind(exchange='logs', queue=queue_name)

print(' [*] Waiting for messages. To exit kernel -> Interrupt')

def callback(ch, method, properties, body):
    print(" [x] %r" % body)

channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

try:
    channel.start_consuming()
except KeyboardInterrupt:
    # Exit gracefully
    connection.close()

## <font color='dimgrey'> Testing </font> 
<t><font color='dimgrey'> We will run publisher code from this notebook and we will create two worker nodes for consuming </font> 

In [4]:
#publisher
# Emit_logs
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='logs', exchange_type='fanout')
def sendQueue():
    try:
        while True:
            # keep taking input till kernel is interrupted
            message = input("Enter new task")
            channel.basic_publish(exchange='logs', routing_key='', body=message)
            print(" [x] Sent %r" % message)
    except KeyboardInterrupt:
        # Exit gracefully
        connection.close()
sendQueue()

Enter new taskm .
 [x] Sent 'm .'
Enter new taskm ..
 [x] Sent 'm ..'
Enter new taskm ...
 [x] Sent 'm ...'
Enter new taskm ...
 [x] Sent 'm ...'


> Both the worker node recieved both the messages. In the next tutorial we try to subscribe to a subset of messages 